**POZNÁMKA: Tento notebook je určený pre platformu Google Colab. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys

# install deps for explain manually, since pdpbox requires
# an ancient version of matplotlib as a dep
!{sys.executable} -m pip install --no-deps pdpbox lime eli5
!{sys.executable} -m pip install class_utils@git+https://github.com/michalgregor/class_utils.git
#!{sys.executable} -m pip install class_utils[explain]@git+https://github.com/michalgregor/class_utils.git

!{sys.executable} -m pip install xgboost

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, KBinsDiscretizer
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from class_utils import Explainer
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

try:
    import google.colab
    COLAB_MODE = True
except:
    COLAB_MODE = False

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
DATA_HOME = "https://github.com/michalgregor/ml_notebooks/blob/main/data/{}?raw=1"

from class_utils.download import download_file_maybe_extract
download_file_maybe_extract(DATA_HOME.format("titanic.zip"), directory="data/titanic")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

## Interpretácia modelov

Existuje množstvo aplikácií, kde nestačí natrénovať model a použiť ho na výpočet predikcií. Musíme byť schopné model interpretovať a vysvetliť, prečo bol predikovaný daný výstup. Interpretovateľnosť je nevyhnutným predpokladom bezpečného a dôveryhodného strojového učenia a umelej inteligencie – pomáha verifikovať, že systém nie je zaujatý a že jeho predikcie nie sú založené na chránených atribútoch ako je napríklad rasa. V niektorých krajinách interpretovateľnosť dokonca vyžaduje zákon: V EU napríklad platí, že kedykoľvek robí automatický systém rozhodnutia a ľuďoch, majú právo na vysvetlenie.

Existujú modely, ktoré sú samy o sebe do určitej miery interpretovateľné. Dobrým príkkladom sú rozhodovacie stromy: strom je v podstate len súborom pravidiel. Dá sa vizualizovať a prečítať. Pri väčšine modelov to však možné nie je. Aj v prípade rozhodovacích stromov je to tým ťažšie, čím sú väčšie a akonáhle zostavíme z rozhodovacích stromov komisiu, už to prakticky nie je možné vykonať.

V tomto notebook-u ukážeme niekoľko generických metód, ktoré pomáhajú interpretovať predikcie ľubovoľných modelov. Začneme znovu tým, že si načítame a predspracujeme dátovú množinu [Titanic](https://www.kaggle.com/c/titanic).



In [ ]:
df = pd.read_csv("data/titanic/train.csv")
df_train, df_test = train_test_split(df, test_size=0.25,
                     stratify=df["Survived"], random_state=4)

categorical_inputs = ["Pclass", "Sex", "Embarked"]
numeric_inputs = ["Age", "SibSp", 'Parch', 'Fare']
output = "Survived"

class_names = [ "died", "survived"]

In [ ]:
input_preproc = make_column_transformer(
    (make_pipeline(
        SimpleImputer(strategy="most_frequent"),
        OrdinalEncoder()),
     categorical_inputs),
    
    (make_pipeline(
        SimpleImputer(),
        StandardScaler()),
     numeric_inputs)
)

In [ ]:
X_train = input_preproc.fit_transform(df_train[categorical_inputs+numeric_inputs])
Y_train = df_train[output].values.reshape(-1)

X_test = input_preproc.transform(df_test[categorical_inputs+numeric_inputs])
Y_test = df_test[output].values.reshape(-1)

Z pipeline objektu extrahujeme transformátory na doplnenie chýbajúcich hodnôt. Sú dôležité: pomocou nich budeme nižšie konštruovať vysvetľovač.



In [ ]:
categorical_imputer = input_preproc.transformers_[0][1][0]
numeric_imputer = input_preproc.transformers_[1][1][0]

Ďalej na dátach natrénujeme XGBoost mode a určíme správnosť na testovacích dátach, aby sme sa presvedčili, že všetko korektne funguje.



In [ ]:
model = XGBClassifier()
model.fit(X_train, Y_train)

In [ ]:
y_test = model.predict(X_test)
accuracy_score(Y_test, y_test)

Vytvoríme vysvetľovač: pomocný objekt, ktorý nám umožní vytvoriť vysvetlenia. Pri konštrukcii vysvetľovača je kľúčové použiť **rovnaký spôsob doplnenia chýbajúcich hodnôt**  ako sa používa pri tréningu nášho modelu. Inak budeme vysvetľovať iné vzorky, než aké by model bežne videl.



In [ ]:
explainer = Explainer(
    model, df_train,
    categorical_inputs,
    categorical_imputer,
    numeric_inputs,
    numeric_imputer,
    input_preproc,
    class_names
)

### Permutačná významnosť

Prvá vec, ktorú by mohlo byť užitočné poznať je relatívny vplyv jednotlivých príznakov na predikciu (t.j. významnosť príznakov). Jeden dobre známy spôsob ako významnosť príznaku vypočítať, je permutovať jeho stĺpec v dátovej množine (t.j. náhodne pomiešať poradie hodnôt v stĺpci) a sledovať, ako to ovplyvní predikcie. Ak sa zmenia veľmi výrazne, príznak má pravdepodobne pre predikciu kľúčový význam. Ak sa zmenia len málo alebo vôbec, príznak má pravdepodobne zanedbateľný význam.



In [ ]:
perm = explainer.permutation_importance(df_test, Y_test)

V dátovej množine Titanic sa napríklad stĺpec "Sex" (pohlavie) zdá byť zďaleka najvýznamnejší. Indikuje to, že muži a ženy mali zrejme podstatne odlišnú šancu prežiť.

### Grafy čiastkových závislostí

Aby sme preskúmali vplyv príznaku na predikciu podrobnejšie, môžeme použiť tzv. grafy čiastkových závislostí. Vytvoriť sa dajú tak, že sa systematickým spôsobom mení hodnota jedného príznaku a sleduje sa, aký to má vplyv na výsledné predikcie. Pozrime sa, aká je napríklad čiastková závislosť predikcií od stĺpca "Sex" (pohlavie). Pripomeňme si, že predikujeme, či sa osoba zachránila (1) alebo nie (0). Kladné čísla teda vyjadrujú, že príznak zvyšuje šancu prežitia a záporné, že zvyšuje šancu smrti.



In [ ]:
explainer.pdp_plot(df_test, "Sex")

Je zrejmé, že čo sa týka nášho klasifikátora, mužské pohlavie môže podstatne znížiť šancu na prežitie. Medzi mužmi je však omnoho väčší rozptyl než medzi ženami – to znamená, že aspoň pre niektorých mužov môže ešte stále existovať celkom dobrá šanca sa zachrániť.

Ak preskúmame PD graf pre "Fare" (cestovné), mali by sme vidieť kladnú závislosť: vyššie cestovné vo všeobecnosti znamená vyššiu šancu na prežitie.



In [ ]:
explainer.pdp_plot(df_test, "Fare")

Grafy nemusia byť monotónne. Pre vek je napríklad situácia trochu zložitejšia – hoci sa nedá vylúčiť, že to je kvôli šumu v dátach: nejde až o taký výrazný efekt.



In [ ]:
explainer.pdp_plot(df_test, "Age")

### LIME: Lokálne vysvetlenia

Napokon by nás mohli zaujímať ešte aj lokálne vysvetlenia: keď dostaneme konkrétnu vzorku, budeme možno chcieť vedieť, aký vplyv mal každý z príznakov na predikciu. Existuje metóda, ktorá sa nazýva LIME (Local Interpretable Model-agnostic Explanations) a ktorá poskytuje tento typ vysvetlení tak, že okolie predikcie aproximuje lineárnym modelom. Vďaka tomu je tento prístup nezávislý od modelu: funguje s ľubovoľným typom.

Pri experimentovaní s metódou LIME si vyberieme určitú konkrétnu vzorku z dátovej množiny a necháme si ju vysvetliť. Uvidíme, ktoré príznaky majú kladný a ktoré záporný vplyv na určitú predikciu a aká je veľkosť toho vplyvu.



In [ ]:
exp = explainer.explain(df_test.iloc[2])
exp.show_in_notebook(show_all=True, colab_mode=COLAB_MODE)

In [ ]:
exp.as_pyplot_figure()
plt.show()